# Discharge Demo - Using the ArcGIS Python API
This notebook reviews the workflow for extracting data and generating a hydrograph for catchments nearest a user defined point. The pseudocode for this operation is as follows:
1. Get the user point, as lat/long coordinates (Alternatively, geocode address)
* Identify the HUC8 in which it is located
 * Search AGOL for a watershed boundary datatset
 * Retrive the HUC8 layer from the results
 * Query the HUC8 intersecting the user point and report its HUC8 value
* Extract all NWIS gage site locations for the selected HUC8
 * Form and send request to NWIS Service endpoint
 * Handle response to return coordinates of active gage sites in identified HUC8
 * Convert coordinates to feature geometry to feature set to feature layer
* Identify the gage site feature found nearest the user point
* Extract discharge data for selected gage site
 * Form and send request to NWIS Service endpoint
 * Handle response to build dataframe from discharge data
* Plot discharge hydrograph

In [ ]:
#Import and authenticate the GIS module
from arcgis import GIS
try:
    gis = GIS('pro')
except:
    gis = GIS()

Construct a Point object from the user coordinates
 * References: 
  * https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#
  * https://developers.arcgis.com/python/api-reference/arcgis.geometry.html#point

In [ ]:
# Set user coordinates
userLat = 36.0010
userLng = -78.9991

In [ ]:
# Import the arcgis API objects needed
from arcgis import geometry as geom

In [ ]:
#Create the point object
pt = geom.Point({"x" : userLng, 
                 "y" : userLat, 
                 "spatialReference" : {"wkid" : 4326}
           })
print(pt.is_valid())

### Task 1. Finding Watershed

In [ ]:
#Search for a dataset of HUCs 
search_results = gis.content.search('Watershed Boundaries owner:Federal_User_Community',
                                    outside_org=True)
search_results

In [ ]:
item = search_results[0]
item

In [ ]:
theID = item.id
theID

In [ ]:
#For future reference, retrieve Watershed from its item ID
item = gis.content.get('4c08f2e2b13741da96ad4a8f6aa5e36a')

In [ ]:
layers = item.layers
for layer in layers:
    print (layer.properties.name, layer.properties.type)

Layers: https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

In [ ]:
huc8 = layers[4]
for fld in huc8.properties.fields:
    print (fld.name)

In [ ]:
#from arcgis.features import FeatureLayer
print(huc8.url)

https://developers.arcgis.com/labs/rest/query-a-feature-layer/

https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer/4/query?where=1%3D1&text=&objectIds=&time=&geometry=%7B%0D%0A%22x%22+%3A+-78.15%2C+%22y%22+%3A+36.00%2C+%22spatialReference%22+%3A+%7B%22wkid%22+%3A+4326%7D%0D%0A%7D&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=html

In [ ]:
strRest = 'https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer/4/query?where=1%3D1&text=&objectIds=&time=&geometry=%7B%0D%0A%22x%22+%3A+-78.15%2C+%22y%22+%3A+36.00%2C+%22spatialReference%22+%3A+%7B%22wkid%22+%3A+4326%7D%0D%0A%7D&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=html'

In [ ]:
strItems = strRest.split("&")
strDict = {}
for item in strItems[1:]:
    k,v = item.split('=')
    strDict[k]=v

In [ ]:
strDict.keys()

In [ ]:
strDict['geometry']

Create the point used to select the HUC8

https://developers.arcgis.com/python/api-reference/arcgis.geometry.html?highlight=geometry#module-arcgis.geometry

In [ ]:
huc8.properties.capabilities

https://developers.arcgis.com/python/api-reference/arcgis.geometry.filters.html

In [ ]:
query_result = huc8.query(where='1=1',
                          geometry_filter=geom.filters.intersects(pt))

In [ ]:
query_result.spatial_reference

In [ ]:
query_result.sdf.shape

In [ ]:
sdf = query_result.sdf

In [ ]:
sdf.HUC8